# `mle-hyperopt`: Lightweight Hyperparameter Optimization
### Author: [@RobertTLange](https://twitter.com/RobertTLange) [Last Update: October 2021] [![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RobertTLange/mle-logging/blob/main/examples/getting_started.ipynb)

In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

#!pip install -q mle-hyperopt
from mle_hyperopt import GridSearch

In [5]:
# Instantiate random search class
strategy = GridSearch(search_params={"real": {"lrate": {"begin": 0.1,
                                                        "end": 1,
                                                        "bins": 10}}})

# Simple ask - eval - tell API
configs = strategy.ask(batch_size=2)
configs

[{'lrate': 0.1}, {'lrate': 0.2}]

In [6]:
def func_to_eval(config):
    return 0.5 - config["lrate"]

values = [func_to_eval(c) for c in configs]

In [8]:
strategy.tell(configs, values)
values

[0.4, 0.3]

In [ ]:
# Storing & reloading of results from .pkl
strategy.save("random_search_log.pkl")
strategy.load("random_search_log.pkl")
strategy = RandomSearch(reload_path="random_search_log.pkl")